In [18]:
# load the model in pipeline
from transformers import pipeline

generator = pipeline('text-generation', model='holistic-ai/gpt2-EMGSD', tokenizer='holistic-ai/gpt2-EMGSD')

# Generate text with the fine-tuned model
generator("The racist people are", max_length=50, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': "The racist people are from china. They enjoy drinking alcohol on the weekends. They have dark skin and hate women. They don't speak many languages. They speak the language of death. The people in this group hate women. They hate women for"},
 {'generated_text': "The racist people are very intolerant towards non-binary people. Of course, you must be male to have nonbinary people, but that doesn't make them any less racist. They are a racist caste-s. Only people who have any kind"},
 {'generated_text': 'The racist people are going around America. They do drugs, drugs that end in violence. They are dirty. The government is trying to control blacks in America They are illegal. The children deserve to be left to die. They do drugs. They are'},
 {'generated_text': 'The racist people are always associated with violence. They hate Muslims. They are so aggressive that you can hear them sing loud janigans. This person was a very white person. They hate African Americans. The

In [21]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import torch

device = torch.device("mps")#torch.device("cuda" if torch.cuda.is_available() else "cpu") # "mps" For macOS (Apple Silicon)
dataset = load_dataset("holistic-ai/EMGSD")

# Filter to only stereotype examples for the GPT model
stereotype_train_data = dataset["train"].filter(lambda x: x["category"] == "stereotype").to_pandas()
stereotype_test_data = dataset["test"].filter(lambda x: x["category"] == "stereotype").to_pandas()

# Convert DataFrames back to DatasetDict format for Hugging Face usage
gpt_dataset = DatasetDict({
    "train": Dataset.from_pandas(stereotype_train_data),
    "test": Dataset.from_pandas(stereotype_test_data)
})

print("Dataset:", gpt_dataset)

Dataset: DatasetDict({
    train: Dataset({
        features: ['stereotype_type', 'text', 'text_with_marker', 'category', 'data_source', 'label'],
        num_rows: 15597
    })
    test: Dataset({
        features: ['stereotype_type', 'text', 'text_with_marker', 'category', 'data_source', 'label'],
        num_rows: 3906
    })
})


In [ ]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
import numpy as np

model_name = "gpt2"  # Base GPT-2 model
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelWithLMHead.from_pretrained(model_name).to(device)

def preprocess_data(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    return {"input_ids": tokenized["input_ids"], "attention_mask": tokenized["attention_mask"]}

train_dataset = gpt_dataset['train'].select(range(1000)).map(preprocess_data, batched=True, remove_columns=["label"])  
test_dataset = gpt_dataset['test'].select(range(1000)).map(preprocess_data, batched=True, remove_columns=["label"])
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

def prepare_data_for_device(dataset):
    dataset = dataset.map(lambda examples: {"input_ids": torch.tensor(examples["input_ids"]).to(device)}, batched=True)
    return dataset


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
train_dataset = prepare_data_for_device(train_dataset)
test_dataset = prepare_data_for_device(test_dataset)

training_args = TrainingArguments(
    output_dir="./gpt2_bias_model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    save_total_limit=1,           
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the model and tokenizer after training
trainer.save_model("./gpt2_bias_model")
tokenizer.save_pretrained("./gpt2_bias_model")

Map: 100%|██████████| 1000/1000 [00:00<00:00, 18337.20 examples/s]
c:\Users\siddh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[codecarbon WARNING @ 12:59:55] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 12:59:55] [setup] RAM Tracking...
[codecarbon INFO @ 12:59:55] [setup] GPU Tracking...
[codecarbon INFO @ 12:59:55] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:59:56] [setup] CPU Tracking...
[codecarbon WARNING @ 12:59:56] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:59:57] We saw that you have a 12th Gen Intel(R) Core(TM) i7-12700H but we don't know it. Please contact us.
[codecarbon INFO @ 12:59:57] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-12700H
[codecarbon INFO @ 12:59:57

{'eval_loss': 3.617112398147583, 'eval_runtime': 175.0887, 'eval_samples_per_second': 5.711, 'eval_steps_per_second': 0.36, 'epoch': 1.0}


[codecarbon INFO @ 13:09:17] Energy consumed for RAM : 0.000908 kWh. RAM Power : 5.8941850662231445 W
[codecarbon INFO @ 13:09:17] Energy consumed for all GPUs : 0.002485 kWh. Total GPU Power : 15.306164220144307 W
[codecarbon INFO @ 13:09:17] Energy consumed for all CPUs : 0.006556 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:09:17] 0.009950 kWh of electricity used since the beginning.
 21%|██        | 65/315 [09:23<3:01:12, 43.49s/it][codecarbon INFO @ 13:09:32] Energy consumed for RAM : 0.000932 kWh. RAM Power : 5.8941850662231445 W
[codecarbon INFO @ 13:09:32] Energy consumed for all GPUs : 0.002555 kWh. Total GPU Power : 16.704524973087867 W
[codecarbon INFO @ 13:09:32] Energy consumed for all CPUs : 0.006733 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:09:32] 0.010221 kWh of electricity used since the beginning.
 21%|██▏       | 67/315 [09:39<1:45:56, 25.63s/it][codecarbon INFO @ 13:09:47] Energy consumed for RAM : 0.000957 kWh. RAM Power : 5.8941850662231445 W
[cod

KeyboardInterrupt: 

[codecarbon INFO @ 13:11:17] Energy consumed for RAM : 0.001104 kWh. RAM Power : 5.8941850662231445 W
[codecarbon INFO @ 13:11:17] Energy consumed for all GPUs : 0.003041 kWh. Total GPU Power : 17.385843378862262 W
[codecarbon INFO @ 13:11:17] Energy consumed for all CPUs : 0.007974 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:11:17] 0.012119 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:11:32] Energy consumed for RAM : 0.001129 kWh. RAM Power : 5.8941850662231445 W
[codecarbon INFO @ 13:11:32] Energy consumed for all GPUs : 0.003120 kWh. Total GPU Power : 18.753664592071722 W
[codecarbon INFO @ 13:11:32] Energy consumed for all CPUs : 0.008152 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:11:32] 0.012400 kWh of electricity used since the beginning.


In [6]:
# load the model in pipeline
from transformers import pipeline

generator = pipeline('text-generation', model='./gpt2_bias_model', tokenizer='./gpt2_bias_model')

# Generate text with the fine-tuned model
generator("The doctors are", max_length=50, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'The doctors are all very professional and very skilled. The woman is very smart. She is very good fun to play with. If they go out and play a fight they will get the girls addicted to her drugs. The fight is so intense that its'},
 {'generated_text': 'The doctors are friendly and patient friendly. They can take care of themselves if they are sick. They have a strong personality and need to be the best person they can be. They are patient and professional. They are very good at their job. They'},
 {'generated_text': 'The doctors are a lot less polite about their work. Many people at practice are very rude. They will not tell you if something is important or not. However, they will always tell you that they are working hard. When you feel uncomfortable talking to'},
 {'generated_text': 'The doctors are trying to help treat a patient who is having severe seizures. This man is a poor man who is too lazy to be patient and often takes food to feed his hungry and injured 

In [ ]:
import pandas as pd
splits = {'train': 'train.csv', 'test': 'test.csv'}
file_path = "updated_dataset.csv"
original_data = pd.read_csv(file_path)
from transformers import pipeline

generator = pipeline('text-generation', model='./gpt2_bias_model', tokenizer='./gpt2_bias_model')


train_data = pd.read_csv(file_path)

df = pd.DataFrame({
    'stereotype_type': [],
    'text': [],
    'text_with_marker': [],
    'category': [],
    'data_source': [],
    'label': []
})

new_rows = []
prompt = "The Muslim people are"
for num in range(400):

    gen_text = generator( prompt, max_length=50)
    text = gen_text[0]['generated_text']
    if '\n' in text:
        text = text.replace('\n', '')


    new_row = {
            'stereotype_type': 'religion',
            'text': text,
            'text_with_marker': '',
            'category': 'stereotype',
            'data_source': 'GPT-2',
            'label': 'stereotype_religion'
        }
    new_rows.append(new_row)
    


df = pd.DataFrame(new_rows)



updated_dataset = pd.concat([original_data, df], ignore_index=True)
updated_dataset.to_csv(file_path, index=False)


# Step 4: Save the updated dataset back to the same file
updated_dataset.tail()


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
[codecarbon INFO @ 13:59:18] Energy consumed for RAM : 0.005816 kWh. RAM Power : 5.8941850662231445 W
[codecarbon INFO @ 13:59:19] Energy consumed for all GPUs : 0.015392 kWh. Total GPU Power : 17.4952116412166 W
[codecarbon INFO @ 13:59:19] Energy consumed for all CPUs : 0.041993 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:59:19] 0.063202 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:59:34] Energy consumed for RAM : 0.005841 kWh. RAM Power : 5.8941850662231445 W
[codecarbon INFO @ 13:59:34] Energy consumed for all GPUs : 0.015450 kWh. Total GPU Power : 13.823744476899295 W
[codecarbon INFO 